In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange
from scipy.sparse import csr_matrix, vstack
import scipy

In [2]:
raw_train = np.load("/media/hdd01/sklee/CA1_clust4-60/data/stim_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_rep1000_stimseed1_set3.npy")
raw_train = raw_train[raw_train[:,2] <= 10000]
print(raw_train.shape)

rep = 1000
pick_idx = np.arange(1,(rep-1)*2,2)
start_idx = np.zeros((rep+1))
raw_start_idx = np.where(np.diff(raw_train[:,2]) <= -9000)[0][pick_idx] + 1
start_idx[1:-1] += raw_start_idx
start_idx[-1] = raw_train.shape[0]
start_idx = start_idx.astype(int)
print(start_idx)
for i in tnrange(rep):
    raw_train[start_idx[i]:start_idx[i+1],2] += i*10000
    
raw_train = np.delete(raw_train, np.where(np.isnan(raw_train) == True)[0], 0)
E_train = raw_train[np.where(raw_train[:,0]>0)]
I_train = raw_train[np.where(raw_train[:,0]<0)]
print(E_train.shape)
print(I_train.shape)

(25485065, 3)
[       0    25374    50745 ... 25434131 25459582 25485065]


<ipython-input-2-740df0803466>:13: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(rep):


  0%|          | 0/1000 [00:00<?, ?it/s]

(11413317, 3)
(14071748, 3)


In [3]:
E_neural_list = []
I_neural_list = []
bin_size = 1/5
bin_no = 50000

E_count_total = 0
I_count_total = 0
E_no = 2000
I_no = 200

for i in tnrange(rep):
    E_neural = np.zeros((bin_no, E_no))
    I_neural = np.zeros((bin_no, I_no))
    
    bin_edges = np.arange(np.around(i*bin_size*bin_no, 1),
                          np.around((i+1)*bin_size*bin_no+bin_size, 1), bin_size)
    if bin_edges.shape[0] > 50001:
        bin_edges = np.delete(bin_edges, -1, 0)
    elif bin_edges.shape[0] < 50001:
        bin_edges = np.append(bin_edges, bin_edges[-1] + 0.2, 0)
    E_counts, E_edges = np.histogram(E_train[:,2], bin_edges)
    I_counts, I_edges = np.histogram(I_train[:,2], bin_edges)

    E_count = 0
    I_count = 0
    
    for j in range(E_counts.shape[0]):
        bin_count = E_counts[j]
        for k in range(bin_count):
            syn = int(E_train[E_count,1])
            E_neural[j,syn] += 1
            E_count += 1
            
    for j in range(I_counts.shape[0]):
        bin_count = I_counts[j]
        for k in range(bin_count):
            syn = int(I_train[I_count,1])
            I_neural[j,syn] += 1
            I_count += 1
    
    E_count_total += E_count
    I_count_total += I_count
    
    E_neural = csr_matrix(E_neural)
    I_neural = csr_matrix(I_neural)
    E_neural_list.append(E_neural)
    I_neural_list.append(I_neural)
    
print(E_train.shape[0])
print(E_count_total)
print(I_train.shape[0])
print(I_count_total)

<ipython-input-3-08bf6144159c>:11: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(rep):


  0%|          | 0/1000 [00:00<?, ?it/s]

11413317
11414345
14071748
14072761


In [5]:
E_neural_all = scipy.sparse.vstack(E_neural_list)
I_neural_all = scipy.sparse.vstack(I_neural_list)

In [6]:
#scipy.sparse.save_npz("/media/hdd01/sklee/CA1_clust4-60/data/Espikes_neural_set3.npz", E_neural_all)
#scipy.sparse.save_npz("/media/hdd01/sklee/CA1_clust4-60/data/Ispikes_neural_set3.npz", I_neural_all)

In [ ]:
for i in tnrange(4):
    idx = i+2

    na = np.load("/media/hdd01/sklee/CA1_clust12-20/data/vdata_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_rep1000_stimseed1_set"+str(idx)+".npy")[:,:50000].flatten()
    nona = np.load("/media/hdd01/sklee/CA1_clust12-20_noNA/data/vdata_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_noDendNa_Er0.5_Ir7.4_random_NR_rep1000_stimseed1_set"+str(idx)+".npy")[:,:50000].flatten()
    diff = na - nona
    #np.save("/media/hdd01/sklee/CA1_clust12-20/data/V_diff_stimseed1_set"+str(idx)+".npy", diff)
    
#plt.plot(diff[:50000])

<ipython-input-10-a62826fc68d3>:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(4):


  0%|          | 0/4 [00:00<?, ?it/s]